In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import resample

protocolMap = {}

def reformatInfoAndPacket(dataframe):
    protocols = dataframe["Protocol"].value_counts().index
    
    for protocol in protocols:
        if protocol not in protocolMap.keys():
            protocolMap[protocol] = len(protocolMap)
    # for info in infos:
    #     if info not in infoMap.keys():
    #         infoMap[info] = len(infoMap)
    dataframe["Protocol"] = dataframe["Protocol"].map(protocolMap)
    if "Website" in dataframe.columns:
        return dataframe[["Time", "Protocol", "Length","Website"]]
    else:
        return dataframe[["Time", "Protocol", "Length"]]
    
def convertWireSharkData(chatGPTcsv,blackboardcsv, linkedIncsv, sizeOfDataFrame = 0):
    chatGPT = pd.read_csv(chatGPTcsv)
    blackboard = pd.read_csv(blackboardcsv)
    linkedIn = pd.read_csv(linkedIncsv)
    sampleSize = min(len(chatGPT),len(blackboard),len(linkedIn))

    blackboardDownsampled = resample(blackboard,replace=False, n_samples=sampleSize,random_state=42)
    linkedInDownsampled = resample(linkedIn,replace=False, n_samples=sampleSize,random_state=42)
    chatGPTDownsampled = resample(chatGPT,replace=False, n_samples=sampleSize,random_state=42)

    chatGPTDownsampled["Website"] = "ChatGPT"
    blackboardDownsampled["Website"] = "Blackboard"
    linkedInDownsampled["Website"] = "Linkedin"
    completeData = pd.concat([chatGPTDownsampled, blackboardDownsampled,linkedInDownsampled])
    numericData = reformatInfoAndPacket(completeData) 
    
    if sizeOfDataFrame == 0:
        sizeOfDataFrame = len(numericData)
    downSampledData = resample(numericData,replace=False, n_samples=sizeOfDataFrame,random_state=42)
    print(downSampledData)
    return downSampledData,blackboard, chatGPT, linkedIn


    


In [5]:
data,blackboard, chatGPT, linkedIn = convertWireSharkData("lakeData/train/chatdata.csv","lakeData/train/blackboardData.csv","lakeData/train/linkedindata.csv",7000 )
blackboard
print(protocolMap)

           Time  Protocol  Length     Website
2307  70.712969         1      87     ChatGPT
670   12.668260         1      67     ChatGPT
1061  32.229970         1      83     ChatGPT
2384  82.296727         0    1494    Linkedin
416    2.061912         2     529    Linkedin
...         ...       ...     ...         ...
174    1.234227         0    1514  Blackboard
1538  71.180707         0      56    Linkedin
2092  79.571787         2     235    Linkedin
1651  18.730526         0      66  Blackboard
543    2.321296         0    1514  Blackboard

[7000 rows x 4 columns]
{'TCP': 0, 'QUIC': 1, 'TLSv1.2': 2}
